<a href="https://colab.research.google.com/github/gskumlehn/maltese-or-poodle/blob/main/mobilenet_maltese_or_poodle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Updates database from github repository

In [3]:
!rm -rf maltese-or-poodle
!git clone https://github.com/gskumlehn/maltese-or-poodle.git

Cloning into 'maltese-or-poodle'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (993/993), done.
remote: Compressing objects: 100% (989/989), done.
remote: Total 993 (delta 13), reused 970 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (993/993), 32.16 MiB | 36.07 MiB/s, done.
Resolving deltas: 100% (13/13), done.


Creates two MobileNet models that share the same architecture and pre-trained weights but operate on different input layers:

model: Processes raw/original input data.

augmented_data_model: Processes augmented versions of the input data.

In [4]:
import keras
from keras.applications import MobileNet

model=MobileNet(weights='imagenet', include_top=False)
augmented_data_model=MobileNet(weights='imagenet', include_top=False)

<ipython-input-4-145d36900d23>:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model=MobileNet(weights='imagenet', include_top=False)


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


<ipython-input-4-145d36900d23>:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  augmented_data_model=MobileNet(weights='imagenet', include_top=False)


Global average pooling is applied to extract compact feature vectors from the outputs of both the original data model and the augmented data model.

In [5]:
from keras.layers import GlobalAveragePooling2D

x=model.output
x=GlobalAveragePooling2D()(x)

augmented_data_x=augmented_data_model.output
augmented_data_x=GlobalAveragePooling2D()(augmented_data_x)

Creates two separate models for processing original and augmented data, with dense layers added for classification

In [6]:
from keras.models import Model
from keras.layers import Dense

x=Dense(50, activation='relu')(x)
preds=Dense(1, activation='sigmoid')(x)

augmented_data_x=Dense(50, activation='relu')(augmented_data_x)
augmented_data_preds=Dense(1, activation='sigmoid')(augmented_data_x)

model=Model(inputs=model.input, outputs=preds)
augmented_data_model=Model(inputs=augmented_data_model.input, outputs=augmented_data_preds)

Counts layers that have already been trained and set only new ones for training

In [7]:
not_trainable_layer_count = len(model.layers) -1

for layer in model.layers[:not_trainable_layer_count]:
    layer.trainable=False
for layer in model.layers[not_trainable_layer_count:]:
    layer.trainable=True

for layer in augmented_data_model.layers[:not_trainable_layer_count]:
    layer.trainable=False
for layer in augmented_data_model.layers[not_trainable_layer_count:]:
    layer.trainable=True

The dataset directory, target image size, and batch size are defined for preparing and processing image data in TensorFlow.

In [8]:
import tensorflow as tf

base_dir = "/content/maltese-or-poodle/classes"
image_size = (224, 224)
batch_size = 32

The code loads and splits image data from base_dir into training (80%) and validation (20%) datasets, resizing images and batching them. It also retrieves and prints the class names for reference.

In [9]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

class_names = train_dataset.class_names
print("Classes:", class_names)

Found 558 files belonging to 2 classes.
Using 447 files for training.
Found 558 files belonging to 2 classes.
Using 111 files for validation.
Classes: ['maltese', 'poodle']


Sets up a pipeline for augmenting and normalizing images in the training dataset and normalizing the test dataset. This improves generalization during training and ensures consistent input scaling.

In [10]:
from keras.layers import RandomFlip, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
])

normalization_layer = Rescaling(1./255)

augmented_train_dataset = train_dataset.map(lambda x, y: (data_augmentation(normalization_layer(x)), y))
augmented_test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

 Optimizes data pipelines by enabling prefetching, allowing data loading and preprocessing to run concurrently with model training or evaluation, improving computational efficiency.

In [11]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
augmented_train_dataset = augmented_train_dataset.prefetch(buffer_size=AUTOTUNE)
augmented_test_dataset = augmented_test_dataset.prefetch(buffer_size=AUTOTUNE)

 Bompiles both the original and augmented data models with the Adam optimizer, binary cross-entropy loss, and accuracy as the evaluation metric.

In [12]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
augmented_data_model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

Sets epochs

In [13]:
epochs=70

 Trains the models using the training dataset for a specified number of epochs and evaluates it using the validation dataset after each epoch to monitor its performance.

In [14]:
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset
)

Epoch 1/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.4952 - loss: 0.7867 - val_accuracy: 0.4685 - val_loss: 0.7985
Epoch 2/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.4804 - loss: 0.7869 - val_accuracy: 0.4505 - val_loss: 0.7920
Epoch 3/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4868 - loss: 0.7791 - val_accuracy: 0.4505 - val_loss: 0.7858
Epoch 4/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4588 - loss: 0.7921 - val_accuracy: 0.4685 - val_loss: 0.7799
Epoch 5/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.5097 - loss: 0.7579 - val_accuracy: 0.4685 - val_loss: 0.7746
Epoch 6/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4974 - loss: 0.7506 - val_accuracy: 0.4775 - val_loss: 0.7697
Epoch 7/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.5039 - loss: 0.7481 - val_accuracy: 0.4685 - val_loss: 0.7650
Epoch 8/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.4883 - loss: 0.7626 - val_accuracy: 0.4685 - val_loss:

In [ ]:
augmented_data_history = augmented_data_model.fit(
    augmented_train_dataset,
    epochs=epochs,
    validation_data=augmented_test_dataset,
)

Epoch 1/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - accuracy: 0.4462 - loss: 1.0000 - val_accuracy: 0.3514 - val_loss: 1.0155
Epoch 2/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4388 - loss: 0.9737 - val_accuracy: 0.3423 - val_loss: 1.0036
Epoch 3/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4450 - loss: 0.9419 - val_accuracy: 0.3423 - val_loss: 0.9925
Epoch 4/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.4616 - loss: 0.9287 - val_accuracy: 0.3514 - val_loss: 0.9813
Epoch 5/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.4386 - loss: 0.9322 - val_accuracy: 0.3514 - val_loss: 0.9709
Epoch 6/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4474 - loss: 0.9366 - val_accuracy: 0.3514 - val_loss: 0.9610
Epoch 7/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.4329 - loss: 0.9156 - val_accuracy: 0.3604 - val_loss: 0.9515
Epoch 8/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.4599 - loss: 0.9102 - val_accuracy: 0.3604 - val_loss:

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

aug_loss, aug_accuracy = augmented_data_model.evaluate(x_test, y_test, verbose=0)

print('Test augmented loss:', aug_loss)
print('Test augmented accuracy:', aug_accuracy)

In [ ]:
model.save(f'{class_names[0]}-or-{class_names[1]}-mobilenet.h5')
from google.colab import files
files.download(f'{class_names[0]}-or-{class_names[1]}-mobilenet.h5')

augmented_data_model.save(f'augmented-data-{class_names[0]}-or-{class_names[1]}-mobilenet.h5')
from google.colab import files
files.download(f'augmented-data-{class_names[0]}-or-{class_names[1]}-mobilenet.h5')

Loads test images, preprocesses them, and makes predictions using both the original and augmented data models. It then prints out the predicted class ("poodle" or "maltese") for each test image.

In [ ]:
import os
from keras.preprocessing import image
import numpy as np

test_dir = f'{base_dir}/test'
test_paths = os.listdir(test_dir)
for path in test_paths:
  test_image = image.load_img(f'{test_dir}/{path}', target_size = (224, 224))

  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  test_image = test_image/255

  result = model.predict(test_image)
  augmented_data_result = augmented_data_model.predict(test_image)

  if result[0][0] < 0.5:
      prediction = class_names[0]
  else:
      prediction = class_names[1]

  if augmented_data_result[0][0] < 0.5:
      augmentedData_prediction = class_names[0]
  else:
      augmentedData_prediction = class_names[0]

  print(f'Prediction: {path} is {prediction}')
  print(f'Augmented data prediction: {path} is {augmentedData_prediction}')

Generates a line graph comparing the training and validation accuracy of two models (one trained with original data and the other with augmented data) across multiple epochs, helping to visualize the performance differences between them.

In [ ]:
import matplotlib.pyplot as plt

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

augmented_data_val_loss = augmented_data_history.history['val_loss']
augmented_data_val_accuracy = augmented_data_history.history['val_accuracy']

fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(val_loss)
ax.plot(augmented_data_val_loss)
ax.set_title("Validation Loss")
ax.set_xlabel("Epochs")
ax.legend()

ax2 = fig.add_subplot(122)
ax2.plot(val_accuracy)
ax2.plot(augmented_data_val_accuracy)
ax2.set_title("Validation Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylim(0, 1)
ax2.legend()

plt.show()

In [ ]:
plt.clf()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']
augmented_data_val_loss = augmented_data_history.history['val_loss']
augmented_data_val_accuracy = augmented_data_history.history['val_accuracy']

log_fit_acc = np.polyfit(np.log(epochs), val_loss, 1)
log_fit_aug_acc = np.polyfit(np.log(epochs), augmented_data_val_loss, 1)

log_fit_val_acc = np.polyfit(np.log(epochs), val_accuracy, 1)
log_fit_aug_val_acc = np.polyfit(np.log(epochs), augmented_data_val_accuracy, 1)

# Generate fitted values
log_line_acc = log_fit_acc[0] * np.log(epochs) + log_fit_acc[1]
log_line_aug_acc = log_fit_aug_acc[0] * np.log(epochs) + log_fit_aug_acc[1]

log_line_val_acc = log_fit_val_acc[0] * np.log(epochs) + log_fit_val_acc[1]
log_line_aug_val_acc = log_fit_aug_val_acc[0] * np.log(epochs) + log_fit_aug_val_acc[1]

fig = plt.figure(figsize=(16,4))

ax = fig.add_subplot(121)

ax.plot(epochs, log_line_acc, 'b-', label='Log Fit: Validation loss')
ax.plot(epochs, log_line_aug_acc, 'r-', label='Log Fit: Augmented Validation Loss')
ax.set_title("Validation Loss")
ax.set_xlabel("Epochs")
ax.legend()

ax2 = fig.add_subplot(122)

ax2.plot(epochs, log_line_val_acc, 'b-.', label='Log Fit: Validation Accuracy')
ax2.plot(epochs, log_line_aug_val_acc, 'r-.', label='Log Fit: Augmented Validation Accuracy')
ax2.set_title("Validation Accuracy")
ax2.set_xlabel("Epochs")
ax.legend()

plt.show()
